In [128]:
import torch.nn as nn
import torch.nn.functional as F
import torch

In [129]:
# hyperparameters
batch_size = 9
num_examples = 8
d_in = 8
d_out = 4

In [130]:
# Our input example
X = torch.randn([batch_size,num_examples,d_in])


In [131]:
X

tensor([[[-5.8789e-01, -6.1364e-01,  1.9645e-01, -1.5171e-01, -9.2134e-01,
           1.3028e-01,  7.5514e-01, -1.9827e+00],
         [-1.4045e-01,  9.8617e-01, -5.7384e-01, -1.5369e-01,  8.4808e-01,
          -1.1877e+00,  1.5344e+00, -7.0977e-01],
         [ 1.2088e+00, -1.0237e+00, -1.1391e+00, -3.3154e-01,  3.0497e-01,
           1.9255e-02,  3.6668e-01,  2.9904e-01],
         [ 2.1346e-01,  4.5701e-01, -1.7439e+00, -9.0076e-02, -2.7547e-01,
           2.7862e-01, -1.6901e-01,  7.9641e-01],
         [-1.1560e+00, -1.9099e+00, -7.3260e-01,  2.4711e-01, -7.2001e-01,
           1.6052e+00,  2.9554e-01,  1.3728e+00],
         [ 6.0220e-01, -1.8257e+00,  5.9340e-02,  2.1575e+00, -3.6973e-01,
           8.0904e-01, -3.0706e-01, -5.3579e-02],
         [ 1.0991e+00,  4.3392e-01,  1.9165e+00, -8.7463e-01, -1.2195e+00,
           3.2940e-01,  3.3150e-01, -1.1092e+00],
         [ 8.3750e-01, -2.3653e+00, -4.5249e-01,  1.1195e+00,  7.8909e-01,
          -2.9319e-04, -2.0543e+00,  1.6910e+00]],

In [ ]:
class selfattention(nn.Module):
    def __init__(self,d_in,d_out,device):
       super(selfattention,self).__init__()
       self.d_in = d_in
       self.d_out = d_out
       # Linear has better initialization compared to other ways of implementation I know
       self.device = device
       self.query = nn.Linear(d_in,d_out,device=device) ## d_in,d_out
       self.key = nn.Linear(d_in,d_out,device = device) ## d_in ,d_out
       self.value = nn.Linear(d_in,d_out, device = device) ## d_in,d_out
       self.softmax = nn.Softmax(dim = -1 )
    def forward(self,X):
        qx = self.query(X)
        kx = self.key(X)
        vx = self.value(X)
        # Q @ K.T
        kx = torch.transpose(kx,1,2)
        y = qx @ kx
        y /= (self.d_out)**0.5
        # creating a mask for causal attention
        mask = torch.ones(y.shape, device=self.device)
        mask = torch.triu(mask,diagonal=1)
        mask = mask.bool()
        # applying mask
        y  = torch.masked_fill(y,mask,-torch.inf)
        # applying softmax
        y = self.softmax(y)
        # y = (QK.T)@ V
        y = y @ vx  
        return y

In [133]:
device = "cpu"
attention_head = selfattention(d_in,d_out,device)


In [134]:
context_vector = attention_head(X)

In [135]:
print(context_vector)

tensor([[[-5.2791e-02,  3.5500e-02,  2.2095e-02, -3.1855e-03],
         [-3.5900e-02, -5.4591e-03,  7.4538e-03,  1.6181e-02],
         [-2.4944e-02,  3.8457e-03, -6.0029e-02, -1.4065e-01],
         [-1.7933e-01,  2.1461e-01, -1.4414e-01, -2.5231e-01],
         [-5.2971e-01,  6.7651e-01,  8.8383e-02, -2.4364e-01],
         [-4.3604e-01,  4.4454e-01, -1.6002e-01, -3.3741e-01],
         [-8.3631e-01,  8.7247e-01,  9.2844e-02, -5.7047e-01],
         [-2.7329e-01,  7.9181e-01, -3.6924e-01, -1.7115e+00]],

        [[ 5.0260e-05, -4.3991e-04, -2.3471e-02, -5.7827e-02],
         [-8.8391e-02, -1.2805e-02,  1.9308e-01,  6.4480e-02],
         [-5.2383e-02,  9.3612e-03,  1.0690e-01,  5.6087e-02],
         [-1.2360e-02, -2.9350e-02, -2.8846e-02, -3.7432e-02],
         [-3.0017e-02, -1.4870e-01,  2.3430e-02,  2.7953e-01],
         [-2.6586e-01,  2.8983e-02,  7.2338e-02,  4.7788e-01],
         [-6.1716e-01,  4.9957e-01, -9.0169e-01, -5.1610e-01],
         [-9.6795e-01,  2.4801e-01, -5.1529e-01,  7.3

In [137]:
context_vector.shape

torch.Size([9, 8, 4])